In [1]:
import os

In [2]:
%pwd

'c:\\Users\\dhava\\OneDrive\\Desktop\\mlproject\\mlproject\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\dhava\\OneDrive\\Desktop\\mlproject\\mlproject'

In [6]:
import dagshub
dagshub.init(repo_owner='dhavalpatel9710', repo_name='mlproject', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\dhava\OneDrive\Desktop\mlproject\mlproject\.conda\Lib\site-packages\rich\live.py:231: UserWarning: install
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=6bebc589-c2d7-4f18-80fb-4c5e41bf6510&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=a63df8da9128b345d717fa8a38e9a5a6cbdc95af14eaaaf1e5a5b825102d2b2f




Accessing as dhavalpatel9710

Initialized MLflow to track repo "dhavalpatel9710/mlproject"

Repository dhavalpatel9710/mlproject initialized!

2024/11/11 20:42:27 INFO mlflow.tracking._tracking_service.client: 🏃 View run capable-swan-332 at: https://dagshub.com/dhavalpatel9710/mlproject.mlflow/#/experiments/0/runs/07e215784323459fa16d0f00d8d30b82.
2024/11/11 20:42:27 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dhavalpatel9710/mlproject.mlflow/#/experiments/0.


In [7]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/dhavalpatel9710/mlproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] ="dhavalpatel9710"
os.environ["MLFLOW_TRACKING_PASSWORD"]= "785dc5eb4dffd3123c6c6d4d558bc30d12818d0b"
           

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [9]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH,schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_roots])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri="https://dagshub.com/dhavalpatel9710/mlproject.mlflow"
        )

        return model_evaluation_config

In [15]:
import pandas as pd
import os
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlProject.utils.common import save_json

In [13]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        
        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualitites = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y,predicted_qualitites)

            ## Saving metrics as local
            scores = {"rmse": rmse,"mae": mae,"r2": r2}
            save_json(path = Path(self.config.metric_file_name),data = scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model,"model")




In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-11-11 21:47:20,287: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-11 21:47:20,288: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-11 21:47:20,288: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-11 21:47:20,296: INFO: common: created directory at: artifacts]
[2024-11-11 21:47:20,298: INFO: common: created directory at: artifacts/model_evaluation]
[2024-11-11 21:47:20,450: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/11/11 21:47:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2024/11/11 21:47:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
2024/11/11 21:47:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run polite-cub-558 at: https://dagshub.com/dhavalpatel9710/mlproject.mlflow/#/experiments/0/runs/d3c5dacafdb5487790fc35583fe90ca6.
2024/11/11 21:47:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dhavalpatel9710/mlproject.mlflow/#/experiments/0.
